In [2]:
import pandas as pd
import datatools as dt
import math as m
from random import randint

dir_hackathon = '.'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

Загрузка данных. Колонки "Коды ТН ВЭД ЕАЭС", "Технические регламенты" и "Группа продукции" - категориальные. Нам нужно их обработать и превратить в численные. Применим два подхода - нумерацию и разбиение на много колонок.

In [3]:
all_data_in = pd.read_excel(f'{dir_data_in}/dataset.xlsx', header=0)
all_data_in.columns = ['Номер продукции', 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции', 'Общее наименование продукции', 
                       'ИЛ', 'Заявитель', 'Адрес Заявителя', 'Изготовитель', 'Страна', 'Адрес изготовителя']

all_data_in.head(2)

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ


Обработаем коды ТН ВЭД. Запомним копию данных, которую будем считать валидной. 

In [4]:
all_data_in = dt.preprocess_tnved(all_data_in)
valid = all_data_in.copy()

Для каждого из рандомизируемых столбиков выберем все уникальные даты и будем формировать ошибочные строки, содержащие одно или несколько неверных значений. Для начала озаботимся константами. 


In [5]:
randomized_columns = ['Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции']
max_err_columns = 1
err_count_factor = 0.10
err_count = m.floor(all_data_in.shape[0] * err_count_factor)

Выберем уникальные значения и сгенерируем нужное количество ошибочных строк, замещая случайный столбик из выбранных значением из соответствующей группы

In [6]:
unique = [all_data_in[col].unique() for col in randomized_columns]
for i in range(len(randomized_columns)):
    print(randomized_columns[i])
    print(unique[i][:5])

Коды ТН ВЭД ЕАЭС
['4016999708' '7308909809' '8415900009' '8414594000'
 '6103230000;6104630000;6104230000;6104530000;6105201000;6106200000;6110309900;6104430000;6103430000;6110309100;6104130000;6103109000;6103330000']
Технические регламенты
['ТР ЕАЭС 043/2017 О требованиях к средствам обеспечения пожарной безопасности и пожаротушения'
 'ТР ТС 007/2011 О безопасности продукции, предназначенной для детей и подростков'
 'ТР ТС 012/2011 О безопасности оборудования для работы во взрывоопасных средах'
 'ТР ТС 004/2011 О безопасности низковольтного оборудования'
 'ТР ТС 017/2011 О безопасности продукции легкой промышленности']
Группа продукции
['Узлы пересечения противопожарных преград кабельными изделиями, шинопроводами, герметичными кабельными вводами, муфтами и трубопроводами инженерных систем зданий и сооружений'
 'Противопожарные окна, двери, двери шахт лифтов с нормируемым пределом огнестойкости, ворота, люки, шторы, роллеты, экраны, занавесы'
 'Клапаны противопожарные нормально открытые

Сгенерируем нужное количество строк-ошибок

In [7]:
err_data = pd.DataFrame(columns=valid.columns)
row_count = valid.shape[0]
for err_num in range(err_count):
    source_row_ind = randint(0, row_count-1)
    err_row = valid.iloc[source_row_ind].copy()
    # число ошибок не гарантируетс равным max_err_columns
    for err_ind in range(max_err_columns):
        err_col_ind = randint(0, len(randomized_columns)-1)
        err_col = randomized_columns[err_col_ind]
        #делаем так, чтобы точно была ошибка
        donor_row_ind = randint(0, row_count-1)
        while valid.iloc[donor_row_ind][err_col] == err_row[err_col]:
            donor_row_ind = randint(0, row_count-1)
        err_row[err_col] = valid.iloc[donor_row_ind][err_col]
    err_row["is_valid"] = False
    err_data = err_data.append(err_row)
valid["is_valid"] = True
result_df = pd.concat([valid, err_data])
result_df.head(5)

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя,is_valid
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ,True
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ,True
2,2112 Продукция,8415900009,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Клапаны противопожарные нормально открытые, кл...",Клапаны противопожарные марки КПВ.01(EI60) пря...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu,True
3,50677 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы крышные радиальные дымоудаления с ...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu,True
4,51009 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы радиальные дымоудаления типов ВР-8...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu,True


In [8]:
result_df.tail(5)


,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя,is_valid
25620,54992 Продукция,Not presented,ТР ТС 008/2011 О безопасности игрушек,Платформы,Вагоны-платформы модели 13-9924-01,ИЛ Джек,ООО Сайгак 4,РОССИЯ 412275 БОЛЬШОЙ КАРАЙ САРАТОВСКАЯ ОБЛАСТЬ,Aphyocharax alburnus 7 LTD,ТАИЛАНД,TH 50000 Chiang Mai Muang Chiang Mai,0.0
25209,23567 Продукция,6112411000;6112311000;6112319000;6112419000,NaN,Изделия купальные,Изделия трикотажные купальные первого слоя из ...,ИЛ Коптотирис Адамса,ООО Платяная моль 4,РОССИЯ 404157 КРАСНЫЙ БУКСИР ВОЛГОГРАДСКАЯ ОБЛ...,Mesocricetus brandti 6 LTD,ПОЛЬША,PL 64-800 Greater Poland Ciszewo,0.0
58521,66463 Продукция,8481808199,ТР ТС 010/2011 О безопасности машин и оборудов...,Соединители электрические промышленного назнач...,Арматура промышленная трубопроводная,ИЛ Паразитический оруссус,ООО Сиамская косатка 7,РОССИЯ 658574 КОСТИН ЛОГ АЛТАЙСКИЙ КРАЙ,ООО Каспийская минога 13,РОССИЯ,РОССИЯ 396951 ЛАТНОЕ ВОРОНЕЖСКАЯ ОБЛАСТЬ,0.0
22117,19684 Продукция,6117;6111,"ТР ТС 007/2011 О безопасности продукции, предн...","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",Изделия верхние трикотажные второго слоя для д...,ИЛ Железняк,ООО Гоголь 4,РОССИЯ 393588 ВАРВАРИНО ТАМБОВСКАЯ ОБЛАСТЬ,Endromis versicolora 6 LTD,НОРВЕГИЯ,NO 4370 Rogaland Egersund,0.0
30864,62670 Продукция,Not presented,"ТР ТС 007/2011 О безопасности продукции, предн...",Нагреватели,Изделия трикотажные бельевые для мальчиков и д...,ИЛ Массивная синанодонта,ООО Дикобраз 1,РОССИЯ 155140 МАРКОВО ИВАНОВСКАЯ ОБЛАСТЬ,ООО Хохлатые перепела 15,РОССИЯ,РОССИЯ 619435 ПОРОШЕВО ПЕРМСКИЙ КРАЙ,0.0


Применим категориальное преобразование. Взять сразу готовое нельзя, чтобы не мучитьс потом с несколькими столбиками при внесении ошибок. 

In [9]:
result_df = dt.make_columns_categorical(result_df)
result_df.to_excel(f'{dir_data_out}/dateset_testable.xlsx')


Группа продукции : 2066
Технические регламенты : 72
Коды ТН ВЭД ЕАЭС : 6786
